In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [2]:
energy_df = pd.read_csv('C:/Users/DELL/Downloads/energydata_complete.csv')
energy_df = pd.DataFrame(energy_df)
energy_df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [52]:
# QUESTION 12
X_train, X_test, Y_train, Y_test = train_test_split(energy_df[['T2']], energy_df[['T6']], test_size=0.3, random_state=42)
linear = LinearRegression().fit(X_train, Y_train)
ypred = linear.predict(X_test)
score = r2_score(Y_test, ypred)
print('R^2 : {:.2f}'.format(score))

R^2 : 0.64


In [3]:
# QUESTION 13
energy_df.drop(['date', 'lights'], axis=1, inplace=True)
energy_df.dropna(inplace=True)
X= pd.DataFrame(MinMaxScaler().fit_transform(energy_df), columns=energy_df.columns)
X=X.drop('Appliances', axis=1)
Y = MinMaxScaler().fit_transform(energy_df[['Appliances']])
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
model= LinearRegression()
model.fit(x_train, y_train)
predicted_values = model.predict(x_test)
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, predicted_values)
print('mean absolute error {:.2f}'.format(mae))

In [9]:
# QUESTION 14
rss = np.sum(np.square(y_test - predicted_values))
print('residual sum of squares',round(rss, 2)) 

residual sum of squares 45.35


In [10]:
# QUESTION 15
from sklearn.metrics import mean_squared_error as mse
RMSE = np.sqrt(mse(y_test, predicted_values))
print('root mean square error',round(RMSE, 3)) 

root mean square error 0.088


In [11]:
# QUESTION 16
from sklearn.metrics import r2_score
coefficient_of_determination = r2_score(y_test, predicted_values)
print('coefficient of determination {:.2f}'.format(coefficient_of_determination))

coefficient of determination 0.15


In [32]:
# QUESTION 17
#obtain feature weights in ascending order
weights = pd.Series(model.coef_.reshape(-1), x_train.columns).sort_values()
weights_df = pd.DataFrame(weights).reset_index()
weights_df.columns=['Features', 'linear weights']
weights_df['linear weights'].round(3)
weights_df

,Features,linear weights
0,RH_2,-0.456698
1,T_out,-0.321860
2,T2,-0.236178
3,T9,-0.189941
4,RH_8,-0.157595
5,RH_out,-0.077671
6,RH_7,-0.044614
7,RH_9,-0.039800
8,T5,-0.015657
9,T1,-0.003281


In [42]:
# QUESTION 18
ridge_reg=Ridge(alpha=0.4)
ridge_reg.fit(x_train, y_train)
predicted_valuess=ridge_reg.predict(x_test)
from sklearn.metrics import mean_squared_error as mse
RMSE1 = np.sqrt(mse(y_test, predicted_valuess))
print('RMSE-root mean square error',round(RMSE1, 3)) 

RMSE-root mean square error 0.088


In [39]:
# QUESTION 19
lasso_reg=Lasso(alpha=0.001)
lasso_reg.fit(x_train, y_train)
weights = pd.Series(lasso_reg.coef_, x_train.columns).sort_values()
weights_df = pd.DataFrame(weights).reset_index()
weights_df.columns=['Features', 'lasso weights']
weights_df['lasso weights'].round(3)
weights_df

Lasso(alpha=0.001)

4 non-zero features

In [43]:
# QUESTION 20
predicted_valuess=lasso_reg.predict(x_test)
from sklearn.metrics import mean_squared_error as mse
RMSE2 = np.sqrt(mse(y_test, predicted_valuess))
print('RMSE-root mean square error',round(RMSE2, 3)) 

RMSE-root mean square error 0.094
